In [113]:
# Import libraries
import numpy as np
import pandas as pd

# Import data
frauds = pd.read_csv('data/frauds.csv')
nonfrauds = pd.read_csv('data/nonfrauds.csv')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [137]:
Freshfrauds = pd.read_csv('data/FreshFrauds.csv')

def ToDate(date):
    s = str(date[-4:] + "/" + date[:-5])
    return s

def ToAmount(n):
    return n[1:]

# Remove Na date
Freshfrauds = Freshfrauds[Freshfrauds['Date of Txn'].notna()]

# Remove all inappropriate dates
for date in Freshfrauds['Date of Txn']:
    if type(date) == str:
        if date[-5] != "/":
            Freshfrauds = Freshfrauds[Freshfrauds['Date of Txn'] != date]
            
# Change date to appropriate format and comvert it to datetime format
Freshfrauds['Date of Txn'] = Freshfrauds['Date of Txn'].apply(ToDate)
Freshfrauds['cc_amount'] = Freshfrauds['Amount'].apply(ToAmount)
Freshfrauds['user_transaction_time'] = pd.to_datetime(Freshfrauds['Date of Txn'])

# Modify Columns
Freshfrauds['key'] = Freshfrauds['Supplier Key']
Freshfrauds['transaction_risk_score'] = Freshfrauds['Risk Score']
Freshfrauds = Freshfrauds.drop(['Supplier Name','Customer First Name','Customer Last Name','Merchant','Month of Disputed Txns',
                 'Date Dispute Initated','Comment','Marqeta Dispute Decision','Auth ID','MID','Temporary Credit Amount',
                 'Temporary credit reversed','Logged by','Supplier Key','Amount','Risk Score'], axis=1)

unadded_columns = ['channel','processing_type','ledger_balance','cc_acceptor_state','acceptor_state','cc_acceptor_country',
                   'cardholder_presence','card_presence','partial_approval_capable','pin_present']

for col in unadded_columns:
    Freshfrauds[col] = np.nan
    

    
Freshfrauds = Freshfrauds[['key','user_transaction_time','channel','transaction_risk_score','processing_type','cc_amount',
                          'ledger_balance','cc_acceptor_state','acceptor_state','cc_acceptor_country','cardholder_presence',
                          'card_presence','partial_approval_capable','pin_present']]

# Combine the frauds
df = frauds.append(Freshfrauds)

# Export data
df.to_csv('data/updatedfrauds.csv')

In [146]:
# Import data
frauds = pd.read_csv('data/updatedfrauds.csv')
frauds = frauds.drop(frauds.columns[[0]], axis = 1)
frauds = frauds[frauds['user_transaction_time'].notna()]
nonfrauds = pd.read_csv('data/nonfrauds.csv')

# Convert datetime and label data 
frauds['user_transaction_time'] = pd.to_datetime(frauds['user_transaction_time'], errors='coerce', utc=True)
frauds['date'] = frauds['user_transaction_time'].dt.date

nonfrauds['user_transaction_time'] = pd.to_datetime(nonfrauds['user_transaction_time'])
nonfrauds['date'] = nonfrauds['user_transaction_time'].dt.date

frauds['is_fraud'] = 1
nonfrauds['is_fraud'] = 0

# Sampling for each day by taking 4% of good transactions and 100% of frauds
frauds_date = sorted(list(set(frauds['date'])))[4:]
nonfraud = pd.DataFrame()

for date in frauds_date:
    dftemp = nonfrauds.loc[nonfrauds['date'] == date]
    dftemp = dftemp.sample(frac = 0.04, random_state = 0)
    nonfraud = nonfraud.append(dftemp)

df2 = frauds.append(nonfraud)
df2 = df2.sort_values(by='date')
df2 = df2[df2['date'] >= pd.Timestamp(2018,1,1)]
df2 = df2.drop(['key'], axis=1)
df2.to_csv('data/LabeledTransactions.csv')

In [ ]:
df2